In [ ]:
# Import used libraries
import keras
import cv2
import datetime
import pickle
import json
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, Dropout, Lambda, Cropping2D

In [ ]:
# Load and display driving log
with open('config.json') as config_file:    
    config_data = json.load(config_file)
print("Data path: {}".format(config_data['data_path']))


path = config_data['data_path']
file = config_data['driving_log_file']
driving_log = pd.read_csv(path + file)
driving_log

In [ ]:
# Store all images as a numpy array
nb_images = 100 # len(driving_log)
images = []
labels = []
camera_names     = ('center', 'left', 'right')
# Initalize offsets of the steering angle for center, left and right images
steering_offsets = dict({key:val for key,val in zip(camera_names, [0, 0.2, -0.2])})

if not os.path.exists(path + config_data['pickle_file']):
    for camera in camera_names:
        print("Load '{}' images".format(camera))
        labels.append((driving_log['steering'][0:nb_images].values.reshape(-1,1) + steering_offsets[camera]))
        for row in tqdm(range(0, nb_images), unit=' images'):
            img = cv2.imread(path + str.replace(driving_log[camera][row], ' ', ''))
            images.append(np.array(img))

    for image in images.copy():
        images.append(np.fliplr(image))
    for label in labels.copy():
        labels.append(-1*label)
        
    images  = np.stack(images)#.reshape(-1, 160, 320, 3)
    labels  = np.concatenate(labels)
    image_shape = images.shape[1::]
    images, labels = shuffle(images, labels, random_state=42)
    
    # Split dataset to train, test
    X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.3, random_state=42)
    # Save as pickle file
    print("Save data to file: '{}'.".format(config_data['pickle_file']))
    with open(path + config_data['pickle_file'], 'wb') as f: 
        pickle.dump([X_train, X_test, y_train, y_test], f)
    print("Done.")
else:
    print("Load data from file: '{}'.".format(config_data['pickle_file']))
    with open(path + config_data['pickle_file'], 'rb') as f: 
        X_train, X_test, y_train, y_test = pickle.load(f)
    print("Done.")

In [ ]:
# Visualize labels
plt.plot(labels)
plt.show()

In [ ]:
size_of = lambda x: x.shape[0] * x.shape[1] 
# Visualize random image
print("Number of images: 3x{} (center, left, right)".format(len(driving_log)))

img = images[0,::]
plt.imshow(img)
plt.show()
plt.imshow(np.fliplr(img))
plt.show()


r   = 100.0 / img.shape[1]
dim = (100, int(img.shape[0] * r))
dim = (64, 32)
# perform the actual resizing of the image and show it
resized = cv2.resize(img, dim, interpolation=cv2.INTER_CUBIC)
print(resized.shape)
plt.imshow(resized)
plt.show()
print(size_of(img))
print(size_of(resized))
print("Ratio: {}".format((size_of(img)/size_of(resized))))

In [ ]:
model = Sequential()
model.add(Lambda(lambda x: (x / 255) - 0.5, input_shape=image_shape))
model.add(Cropping2D(cropping=((50,20), (0,0))))
model.add(Conv2D(kernel_size=(3,3), filters=32, activation='relu', strides=2))
model.add(Conv2D(kernel_size=(3,3), filters=32, activation='relu', strides=1))
model.add(Conv2D(kernel_size=(3,3), filters=64, activation='relu', strides=2))
model.add(Conv2D(kernel_size=(3,3), filters=64, activation='relu', strides=1))
model.add(Conv2D(kernel_size=(3,3), filters=128, activation='relu', strides=2))
model.add(Conv2D(kernel_size=(1,7), filters=64, activation='relu', strides=1))
model.add(Conv2D(kernel_size=(1,7), filters=64, activation='relu', strides=1))
model.add(Conv2D(kernel_size=(1,7), filters=64, activation='relu', strides=1))
model.add(Conv2D(kernel_size=(1,7), filters=64, activation='relu', strides=1))
model.add(Conv2D(kernel_size=(1,7), filters=32, activation='relu', strides=1))

model.add(Flatten())
model.add(Dense(10, activation='relu'))
model.add(Dense(1))
model.summary()


In [ ]:
now = datetime.datetime.now
batch_size = 30
epochs = 2

model.compile(loss='mse',
              optimizer='adadelta',
              metrics=['accuracy'])

t = now()
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test))
print('Training time: %s' % (now() - t))
score = model.evaluate(X_test, y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])